## 🧩 **Goal**

Recommend products that are often purchased together — a “customers who bought X also bought Y” system.
We’ll use **association rule mining** (Apriori algorithm), a simple yet powerful technique for transactional data.

---

---

## 🔹 **How it Works**

* Each row represents an **order (basket)**.
* Each column represents a **product**.
* The Apriori algorithm finds **frequent item combinations** (e.g., products bought together).
* The `association_rules` function derives rules like:
  “If Product A is bought, Product B is also likely.”

---

## 🔹 **Interpretation Example**

| Rule                     | Support | Confidence | Lift | Meaning                                                                        |
| ------------------------ | ------- | ---------- | ---- | ------------------------------------------------------------------------------ |
| `{Chai}` → `{Chang}`     | 0.08    | 0.55       | 1.9  | 8% of all orders include both; buying Chai nearly doubles odds of buying Chang |
| `{Tofu}` → `{Soy Sauce}` | 0.06    | 0.62       | 2.3  | Tofu buyers are 2.3× more likely to buy Soy Sauce                              |

---

## 🔹 **Business Meaning**

Physically, these results reveal *cross-selling opportunities*.

* Products often bought together likely serve complementary needs (e.g., coffee + sugar).
* You can **bundle** them or **recommend** one after the other in online or sales systems.
* This insight also helps optimize inventory — knowing which items drive joint demand.

